## Helper_functions

In [2]:
from utilities import *

## Library Loading

In [3]:
from libraries import *

## Data Loading

In [1]:
Pickle_path = '../Pickle_Files/'
csv_path = '../Data/'

In [4]:
with open(Pickle_path+"master_df_noNa.pickle",'rb') as fin:
    master_df = pickle.load(fin)

In [5]:
master_df['hashtag_list'] = master_df['tw_text'].apply(extract_hashtag)

In [6]:
master_df['tweet_dt'] = master_df['tw_time'].apply(get_date)

In [7]:
# len(master_df)
# 860485

master_df_nodup = master_df.drop_duplicates(['tw_text'])

# len(master_df_nodup)
# 333116

In [8]:
df_nodup_usa = master_df_nodup[master_df_nodup['usa_flg']==True]

In [9]:
# df_nodup_usa.groupby(['tweet_dt']).agg({'tw_id':'count'}).tail(5)

,tw_id
tweet_dt,
2020-05-27,3033
2020-05-28,3052
2020-05-29,2730
2020-05-30,1820
2020-05-31,1211


### Lets clean the tweet text for running 
1. Sentiment Analysis
2. Stance Detection wrt Covid -19 
3. EMotion extraction for 11 dimensions
4. COnvert text into high dim vector represenation using BERT sentence encoder

#### Text cleaning

In [10]:
# removes special characters (except commas and spaces), URLs, mentions, and hashtags from a tweet text

df_nodup_usa['twt_clean'] = df_nodup_usa['tw_text'].apply(clean_tweet_text)

### Adding Binary Sentiment Score to tweet using 
- distilbert-base-uncased-finetuned-sst-2-english

### Sentiment Extraction using BERT Model

In [ ]:

### we need tweet list for running tasks like emotion, stance, topic modeling. so lets just make a list
tweets_list = df_nodup_usa['twt_clean'].to_list()


with open(Pickle_path+"tweets_list.pkl", "wb") as f:
    pickle.dump(tweets_list, f)

#Example
sentiment_tokenizer_model = 'distilbert-base-uncased'
sentiment_model = 'distilbert-base-uncased-finetuned-sst-2-english'
result=add_sentiment(tweets_list,sentiment_tokenizer_model,sentiment_model)


with open(Pickle_path+"sentiment_results.pkl", "wb") as f:
    pickle.dump(result, f)


##### Sentiment Analysis done
    - output stored in Pickle_path+"sentiment_results.pkl

### Sentence Encoder to encode sentences into Vectors
    -SentenceTransformer('all-MiniLM-L6-v2')
    -spacy.load('en_core_web_sm')

In [38]:

!python3 -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')
model = SentenceTransformer('all-MiniLM-L6-v2')

data_embeddings = []
for des in tqdm(tweets_list):
    embeddings = model.encode(des,)
    data_embeddings.append(embeddings)
    
embeddings_output = data_embeddings

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 101659/101659 [13:22<00:00, 126.64it/s]


In [40]:

with open(Pickle_path+"embeddings_output_tweets.pkl", "wb") as f:
    pickle.dump(embeddings_output, f)


##### Sentence encoding to 364 dim vectors done
    - Output saved in Pickle_path+"embeddings_output_tweets.pkl"

### Stance Detection Module

In [ ]:
# for this, we can load Python file 
# !python Stance_Detection_Module.py


##### Stance Detection Module finished. 
    - Output avaialble in Pickle_path+"Stance_predictions_list.pkl"

### Emotion Extraction Module

In [52]:
emotion_input_df = pd.DataFrame(tweets_list,columns=['Tweet'])

In [53]:
dummy_cols = ["id","anger","anticipation","disgust","fear","joy","love","optimism","pessimism","sadness","surprise","trust"]
emotion_input_df[dummy_cols] = 0

In [60]:
emotion_input_df.to_csv("modify_data.csv",index=False,sep="|")

In [67]:

with open(Pickle_path+"emotion_input_df.pkl", "wb") as f:
    pickle.dump(emotion_input_df, f)


##### After this, Run the code on serve given my Menguxn on the file generated emotion_input_df.pkl

#### It generates a csv file with emotion scores. 

    -Output available as 
    -emotion_scores_output = pd.read_csv(csv_path+"out_put.csv")


### Hashtag Vector Generation Module
    - We only focus on top 1000 hashtags to ensure vector size in under managabele size

In [302]:
MasterDf[['rt_url','tw_id','hashtag_list']].tail(10)

,rt_url,tw_id,hashtag_list
101649,https://conta.cc/2YGSqk0,1.258358e+18,()
101650,https://twitter.com/nytimes/status/12595110326...,1.259571e+18,()
101651,https://trib.al/BPUuFW4,1.259567e+18,()
101652,https://www.news5cleveland.com/news/local-news...,1.259573e+18,()
101653,https://lizditz.typepad.com/i_speak_of_dreams/...,1.259564e+18,()
101654,http://ow.ly/Jkpn50zAi9c,1.259507e+18,()
101655,https://twitter.com/egheitasean/status/1259488...,1.259503e+18,"(#maca,)"
101656,https://www.wsj.com/articles/the-economic-lock...,1.259135e+18,"(#lalege, #lagov)"
101657,http://f-st.co/kLiCICJ,1.259573e+18,()
101658,https://www.vox.com/2020/5/10/21252583/coronav...,1.259557e+18,()


In [298]:

# Read in your master data DataFrame
master_data_df = MasterDf[:]



# Get the top 10,000 hashtags by frequency
all_hashtags = [hashtag for hashtags in master_data_df['hashtag_list'] for hashtag in hashtags]
top_hashtags = pd.Series(all_hashtags).value_counts()[:1000].index.tolist()



# Filter the hashtag lists in your DataFrame to only include the top 10,000 hashtags
filtered_hashtag_lists = [[hashtag for hashtag in hashtags if hashtag in top_hashtags] for hashtags in master_data_df['hashtag_list']]

# Convert the filtered hashtag lists to a one-hot encoded matrix
mlb = MultiLabelBinarizer()
one_hot_matrix = mlb.fit_transform(filtered_hashtag_lists)

# Create a new DataFrame with the one-hot encoded hashtags
# Create a new DataFrame with a single column of 10,000-element lists of encoded values
encoded_hashtags = []
for row in one_hot_matrix:
    encoded_hashtags.append(row.tolist())

encoded_hashtags_df = pd.DataFrame({'encoded_hashtags': encoded_hashtags})





# df_nodup_usa_hashtgs = pd.concat([master_data_df.reset_index(),encoded_hashtags_df],axis=1)



In [299]:
encoded_hashtags_df

,encoded_hashtags
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...
101654,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
101655,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
101656,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
101657,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
with open(Pickle_path+"encoded_hashtags_df.pkl", "wb") as f:
    pickle.dump(encoded_hashtags_df, f)


#### Hashtag Output saved in 
    - Pickle_path+"encoded_hashtags_df.pkl"

### Topic Modeling Module

In [ ]:
# Preprocess the tweets by removing stop words and tokenizing
stop_words = set(stopwords.words('english'))
tweets_df = pd.DataFrame(tweets_list,columns = ['tw_text'])
tweets_df['tokens'] = tweets_df['tw_text'].apply(lambda x: [w.lower() for w in word_tokenize(x) if w.lower() not in stop_words])

# Convert the tweets into a bag-of-words representation
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(tweets_df['tokens'].apply(lambda x: ' '.join(x)))




# Train an LDA model on the bag-of-words matrix
lda_model = LatentDirichletAllocation(n_components=10, max_iter=100, random_state=42)
lda_model.fit(X)

# Print the top words for each topic
feature_names = vectorizer.get_feature_names()
for topic_idx, topic in enumerate(lda_model.components_):
    print("Topic #%d:" % topic_idx)
    print(" ".join([feature_names[i] for i in topic.argsort()[:-10 - 1:-1]]))

# Assign each tweet a distribution over the topics
topic_distributions = lda_model.transform(X)
tweets_df['topic_distribution'] = list(topic_distributions)

# Map the top topic for each tweet to a semantic category
topic_labels = {
    0: 'politics',
    1: 'public health',
    2: 'vaccination',
    3: 'epidemiology',
    4: 'social distancing',
    5: 'mental health',
    6: 'pandemic response',
    7: 'testing',
    8: 'treatment',
    9: 'communication'
}
tweets_df['top_topic'] = tweets_df['topic_distribution'].apply(lambda x: np.argmax(x))
tweets_df['semantic_category'] = tweets_df['top_topic'].map(topic_labels)



with open(Pickle_path+"Topic_modelling_results.pkl",'wb') as topic_out:
    pickle.dump(tweets_df,topic_out)


#### Topic Modeling output available in 
    - Pickle_path+"Topic_modelling_results.pkl"

In [229]:
### Also saving master dataframe after reindexing 

with open(Pickle_path+"df_nodup_usa.pkl",'wb') as topic_out:
    pickle.dump(df_nodup_usa.reset_index(),topic_out)

In [228]:
!ls ../Data

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Sentiment_score_results.csv out_put.csv


#### Now we have outputs from 
    - Sentiment Analysis at sentiment_results.pkl
    - Emotion Analysis at ../Data/out_put.csv
    - Stance wrt Covid-19 at Stance_predictions_list.pkl
    - Hashtag vector at encoded_hashtags_df.pkl
    - Topic Theme Vectors at Topic_modelling_results.pkl
    - Vector Encodings at embeddings_output_tweets.pkl

#### Master DataFrame is available at 
    - Pickle_path+"df_nodup_usa.pkl"

### Loading outputs from all modules into current session

### 1

In [283]:
sentiment_results = pickle_loader(Pickle_path+"sentiment_results.pkl")

sentiment_results_subset = sentiment_results[['sentiment']]

sentiment_results_subset.head(2)

,sentiment
0,-99.427658
1,-99.571890


### 2

In [247]:
stance_results_tmp = pickle_loader(Pickle_path+"Stance_predictions_list.pkl")

stance_results = []

for block in stance_results_tmp:
    for row in block:
        stance_results.append(row)
        


stance_results_df = pd.DataFrame(stance_results,columns = ["contradiction", "entailment", "neutral"])

stance_output_df.head(2)

,contradiction,entailment,neutral
0,0.999482,0.000022,0.000496
1,0.994533,0.000001,0.005466


### 3

In [282]:
emotion_results = pd.read_csv("../Data/out_put.csv")

In [284]:
emotion_results.head(10)

,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,0.641829,0.149830,0.701071,0.024524,0.029221,0.002634,0.025946,0.069477,0.251814,0.025928,0.005484
1,0.236262,0.244001,0.745701,0.350815,0.003112,0.001413,0.006780,0.094898,0.291147,0.142205,0.005690
2,0.018904,0.533875,0.048713,0.003294,0.832357,0.032280,0.788394,0.007254,0.016247,0.131739,0.121118
3,0.006553,0.206196,0.016599,0.016862,0.287812,0.017970,0.631388,0.014190,0.049942,0.024464,0.234475
4,0.035117,0.149447,0.260748,0.179213,0.005718,0.001923,0.017826,0.173826,0.478952,0.028044,0.004780
5,0.027669,0.474313,0.100259,0.084536,0.020202,0.002475,0.049969,0.026967,0.060703,0.212374,0.014591
6,0.006425,0.757777,0.020197,0.068641,0.148025,0.002728,0.766238,0.015592,0.027163,0.034309,0.081728
7,0.339228,0.438487,0.651328,0.056200,0.021179,0.003116,0.015219,0.027993,0.081596,0.240584,0.010672
8,0.134228,0.190615,0.405692,0.067951,0.019809,0.002650,0.032220,0.207090,0.838388,0.019853,0.003878
9,0.753092,0.044671,0.914730,0.941997,0.004812,0.001419,0.008310,0.091530,0.285891,0.016644,0.004070


### 4

In [254]:
hashtag_results = pickle_loader(Pickle_path+"encoded_hashtags_df.pkl")

hashtag_results.head(2)

,encoded_hashtags
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


### 5

In [255]:
Topic_themes_df = pickle_loader(Pickle_path+"Topic_modelling_results.pkl")

In [279]:
Topic_themes_df_sbset = Topic_themes_df[['topic_distribution','top_topic','semantic_category']]
Topic_themes_df_sbset.head(2)

,topic_distribution,top_topic,semantic_category
0,"[0.025002401429176736, 0.02500000005850058, 0....",4,social distancing
1,"[0.011111892748436235, 0.011112492275769222, 0...",8,treatment


### 6

In [258]:
Sentence_embedding_output = pickle_loader(Pickle_path+"embeddings_output_tweets.pkl")


Sentence_embedding_output_df = pd.DataFrame([Sentence_embedding_output]).T

Sentence_embedding_output_df.columns = ['Sentence_Embedding']

In [273]:
Sentence_embedding_output_df.head(2)

,Sentence_Embedding
0,"[-0.027914412, 0.015142873, 0.015514709, -0.05..."
1,"[-0.004281366, 0.043223437, 0.037251867, 0.038..."


In [295]:

MasterDf = pickle_loader(Pickle_path+"df_nodup_usa.pkl")



### Merge All above dataframe to master df to obtain tweet level Features

In [290]:
Tweet_lvl_master_df = pd.concat([MasterDf,sentiment_results_subset,stance_output_df,emotion_results,\
          hashtag_results,Topic_themes_df_sbset,Sentence_embedding_output_df],axis=1)

In [522]:
tmp_url = 'https://www.cnn.com/2020/05/22/investing/moderna-coronavirus-vaccine-stock-sales/index.html'

Tweet_lvl_master_df[Tweet_lvl_master_df['rt_url']==tmp_url][['rtw_time','tw_time','retweet_cnt','rtw_usr_screen','tw_usr_screen']]

,rtw_time,tw_time,retweet_cnt,rtw_usr_screen,tw_usr_screen
0,Sun May 24 02:09:50 +0000 2020,Sat May 23 23:49:25 +0000 2020,49,s_derrickson,Rschooley
32198,Sat May 23 02:14:50 +0000 2020,Fri May 22 21:56:55 +0000 2020,58,stmichaelseesme,DeanBaker13
37340,Fri May 22 21:54:54 +0000 2020,Fri May 22 20:50:10 +0000 2020,14,MOEInvestigator,ArthurCaplan
37341,Fri May 22 21:54:58 +0000 2020,Fri May 22 21:42:29 +0000 2020,10,nanmac321,BlueSt0rmRising
89366,Sat May 23 00:42:01 +0000 2020,Sat May 23 00:31:56 +0000 2020,8,Avonan,juliabhaber


In [523]:


with open(Pickle_path+"Tweet_lvl_master_df.pkl",'wb') as topic_out:
    pickle.dump(Tweet_lvl_master_df,topic_out)

In [335]:
Org_tweets_df = Tweet_lvl_master_df[['rt_url','tw_id', 'tw_time', 'tw_text', 'tw_usr_screen', 'usr_follow_cnt','tweet_dt', 'twt_clean', 'sentiment', 'contradiction',
       'entailment', 'neutral', 'anger', 'anticipation', 'disgust', 'fear',
       'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust',
       'encoded_hashtags', 'topic_distribution', 'top_topic',
       'semantic_category', 'Sentence_Embedding']]

In [336]:
Org_tweets_df = Org_tweets_df.drop_duplicates(['tw_id'])

In [337]:
Org_tweets_df['Tweet_type'] = 'Org'

In [338]:

with open(Pickle_path+"Org_tweets_df.pkl",'wb') as topic_out:
    pickle.dump(Org_tweets_df,topic_out)

In [339]:
retwt_df = Tweet_lvl_master_df[['rt_url','rtw_id', 'rtw_time', 'rtw_text', 'rtw_usr_screen', 'rusr_follow_cnt', 'twt_clean', 'sentiment', 'contradiction',
       'entailment', 'neutral', 'anger', 'anticipation', 'disgust', 'fear',
       'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust',
       'encoded_hashtags', 'topic_distribution', 'top_topic',
       'semantic_category', 'Sentence_Embedding']]

In [340]:
retwt_df['rtweet_dt'] = retwt_df['rtw_time'].apply(get_date)

retwt_df = retwt_df.drop_duplicates(['rtw_id'])

In [341]:
retwt_df['Tweet_type'] = 'Retweet'

In [342]:

with open(Pickle_path+"retwt_df.pkl",'wb') as topic_out:
    pickle.dump(retwt_df,topic_out)


#### Merge tweet, retweet dataframes to create master df

In [343]:
retwt_df_1 = retwt_df[['rt_url','rtw_id', 'rtw_time', 'rtw_text', 'rtw_usr_screen', 'rusr_follow_cnt',\
        'rtweet_dt', 'twt_clean','sentiment', 'contradiction', 'entailment', 'neutral',
       'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism',
       'pessimism', 'sadness', 'surprise', 'trust', 'encoded_hashtags',
       'topic_distribution', 'top_topic', 'semantic_category',
       'Sentence_Embedding','Tweet_type']]

In [344]:
retwt_df_1.columns = list(Org_tweets_df.columns)

In [ ]:
retwt_df_1['tw_usr_screen'].nunique()

In [345]:

### below dataframe is at tweet_id Level

twt_tetwt_master_df = pd.concat([Org_tweets_df,retwt_df_1],axis=0)


In [346]:


with open(Pickle_path+"twt_tetwt_master_df.pkl",'wb') as topic_out:
    pickle.dump(twt_tetwt_master_df,topic_out)

### Roll Up Data at User Level to create individual user Profiles

In [475]:
df_nodup_usa_hashtgs['usr_follow_cnt'] = df_nodup_usa_hashtgs['usr_follow_cnt'].astype(float)


In [478]:

# Read in your df_nodup_usa_hashtgs DataFrame
df_nodup_usa_hashtgs = twt_tetwt_master_df[:]

# Group by tw_usr_screen and aggregate the columns as specified
grouped_df = df_nodup_usa_hashtgs.groupby('tw_usr_screen').agg({
    'encoded_hashtags': lambda x: [sum(val) for val in zip(*x)],   #### taking sum for hashtags
    'topic_distribution': lambda x: [sum(val)/len(val) for val in zip(*x)],#, ## for rest, taking mean
    'Sentence_Embedding': lambda x: [sum(val)/len(val) for val in zip(*x)],#,
    'sentiment': 'mean',
    'contradiction': 'mean',
    'entailment': 'mean',
    'neutral': 'mean',
    'anger': 'mean',
    'anticipation': 'mean',
    'disgust': 'mean',
    'fear': 'mean',
    'joy': 'mean',
    'love': 'mean',
    'optimism': 'mean',
    'pessimism': 'mean',
    'sadness': 'mean',
    'surprise': 'mean',
    'trust': 'mean',
    'rt_url':"count"
#     ,
#     'usr_follow_cnt':'max'
})



In [24]:
list({    'anger': 'mean',
    'anticipation': 'mean',
    'disgust': 'mean',
    'fear': 'mean',
    'joy': 'mean',
    'love': 'mean',
    'optimism': 'mean',
    'pessimism': 'mean',
    'sadness': 'mean',
    'surprise': 'mean',
    'trust': 'mean'}.keys())

['anger',
 'anticipation',
 'disgust',
 'fear',
 'joy',
 'love',
 'optimism',
 'pessimism',
 'sadness',
 'surprise',
 'trust']

In [482]:
User_level_feat_df = grouped_df.reset_index()

In [ ]:
User_level_feat_df.to_csv(csv_path+"User_profile.csv",index=False)

In [253]:
User_level_feat_df = pd.read_csv(csv_path+"User_profile.csv")

In [254]:

gc.collect()

5867

In [255]:
User_level_feat_df.sort_values(['rt_url'],ascending=False,inplace=True)

In [256]:


User_level_feat_df.reset_index(inplace=True)

In [303]:

df['all_features'] = df.apply(lambda row: [row[col] for col in list_cols] + row['list_col'], axis=1)

NameError: name 'list_cols' is not defined

In [420]:
list_cols = ['contradiction', 'entailment', 'neutral', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']

def concat_lists(row):
    return row['encoded_hashtags'] + \
            row['topic_distribution']+[row[col] for col in list_cols] 

# apply the function to each row
User_level_feat_df['Feature_set'] = User_level_feat_df.apply(concat_lists, axis=1)



In [421]:
User_level_feat_df['hastag_sum'] = User_level_feat_df['encoded_hashtags'].apply(sum)


### Lets run KNN to identify 10 neighbours of each tweet based on semantic similarity 
    - Run KNN on sentence embeddings
    - create a adjacency matrix where we get list of all neighbours which we will use to create our graph edges

In [304]:
User_feature_vector = User_level_feat_df['Feature_set'].to_numpy()

KeyError: 'Feature_set'

In [441]:
User_feature_vector = [np.array(w) for w in User_feature_vector]

In [258]:

# assume your DataFrame is named df
tw_usr_screen_dict = User_level_feat_df.reset_index().set_index('tw_usr_screen')['index'].to_dict()


In [455]:
key_2_idx_hashmap = {}
for key,val in tw_usr_screen_dict.items():
    key_2_idx_hashmap[val] = key

In [442]:

# assuming your array is named 'embedding_output'
n_neighbors = 10
knn = NearestNeighbors(n_neighbors=n_neighbors, metric='euclidean')
knn.fit(User_feature_vector)

# get the indices of the 10 nearest neighbors for each row
distances, indices = knn.kneighbors(User_feature_vector)

# create an adjacency dictionary where the key is the row index and the value is a list of its 10 nearest neighbors


In [457]:
adj_dict = {}
for i in range(len(User_feature_vector)):
    adj_dict[key_2_idx_hashmap[i]] = [key_2_idx_hashmap[w] for w in list(indices[i][1:])]


In [459]:

# compute the cosine similarity between each row and its 10 nearest neighbors
similarity_dict = {}
for i in range(len(User_feature_vector)):
    neighbor_indices = indices[i].flatten()
    row_similarities = []
    for ind in neighbor_indices[1:]:
        row_similarities.append(cosine_similarity([User_feature_vector[i]], [User_feature_vector[ind]])[0][0])
    similarity_dict[key_2_idx_hashmap[i]] = dict(zip([key_2_idx_hashmap[w] for w in list(indices[i][1:])], row_similarities))


In [460]:
with open(Pickle_path+"node_similarity_dict.pkl",'wb') as node_similarity_file:
    pickle.dump(similarity_dict,node_similarity_file)

In [467]:
similarity_dict['thehill']
# {'fox5dc': 0.9946963651017979,
#  'WGNNews': 0.9970530660452035,
#  'KDKA': 0.9856506816647358,
#  'AuthorJLLopez': 0.6319364127360368,
#  'ABC7NY': 0.9764324289051096,
#  'WMCActionNews5': 0.9786821664704085,
#  'ajc': 0.9481427513237088,
#  'wxyzdetroit': 0.9504871245739013,
#  'fox32news': 0.9551514801111138}

{'fox5dc': 0.9946963651017979,
 'WGNNews': 0.9970530660452035,
 'KDKA': 0.9856506816647358,
 'AuthorJLLopez': 0.6319364127360368,
 'ABC7NY': 0.9764324289051096,
 'WMCActionNews5': 0.9786821664704085,
 'ajc': 0.9481427513237088,
 'wxyzdetroit': 0.9504871245739013,
 'fox32news': 0.9551514801111138}

In [490]:
Follower_cnt_dict = df_nodup_usa_hashtgs[['tw_usr_screen','usr_follow_cnt']].set_index('tw_usr_screen').to_dict()['usr_follow_cnt']


In [ ]:
pickle_download(Pickle_path+"Follower_cnt_dict.pkl",Follower_cnt_dict)


### GNN + LSTM Module
    -Inputs:
        Adjacency Matrix
        Node Feature Dictionary
   

In [4]:
master_df = pickle_loader(Pickle_path+"Tweet_lvl_master_df.pkl")

In [5]:
master_df.columns

Index(['index', 'rtw_time', 'rtw_id', 'rtw_text', 'rtw_usr_screen',
       'rusr_follow_cnt', 'retweet_cnt', 'rusa_flg', 'rt_url', 'tw_time',
       'tw_id', 'tw_text', 'tw_usr_screen', 'usr_follow_cnt', 'usa_flg',
       'hashtag_list', 'tweet_dt', 'twt_clean', 'sentiment', 'contradiction',
       'entailment', 'neutral', 'anger', 'anticipation', 'disgust', 'fear',
       'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust',
       'encoded_hashtags', 'topic_distribution', 'top_topic',
       'semantic_category', 'Sentence_Embedding'],
      dtype='object')

In [75]:
import pandas as pd


master_df['tw_time'] = pd.to_datetime(master_df['tw_time'], format="%a %b %d %H:%M:%S %z %Y")



In [27]:
import pandas as pd

# Assuming `master_df` is a pandas DataFrame with the columns you described, as well as a `rt_url` column

# Group the DataFrame by `rt_url`
groups = master_df.groupby('rt_url')


In [539]:
# for name, group in groups:
#     # Sort the group by `tw_time`
#     sorted_group = group.sort_values('tw_time')
#     dd = sorted_group.head(100)
#     break
    

In [8]:

results = []
for name, group in groups:
    # Sort the group by `tw_time`
    sorted_group = group.sort_values('tw_time')
    
    # Compute the cutoff time for the first 60% of the tweets
    cutoff_time = sorted_group.iloc[int(0.6 * len(sorted_group))]['tw_time']
    
    # Select the rows that fall within the first 60% of the tweets
    first_60_percent = sorted_group[sorted_group['tw_time'] <= cutoff_time]
    first_60_percent_unq = first_60_percent.drop_duplicates(['tw_usr_screen'])
    
    # Aggregate the `retweet_cnt`, `rtw_usr_screen`, and `tw_usr_screen` columns of the selected rows
#     first_60_percent_agg = {
#         'rt_url': name,
#         'retweet_cnt_sum_60': first_60_percent_unq['retweet_cnt'].sum(),
#         'rtw_usr_screen_list_60': tuple(set(first_60_percent['rtw_usr_screen'].tolist())),
#         'tw_usr_screen_list_60': tuple(set(first_60_percent['tw_usr_screen'].tolist()))
#     }
    
    # Select all the rows
    all_rows = sorted_group
    all_rows_unq = sorted_group.drop_duplicates(['tw_usr_screen'])
    
    # Aggregate the `retweet_cnt`, `rtw_usr_screen`, and `tw_usr_screen` columns of all the rows
    first_60_percent_agg = {
                'rt_url': name,
        'retweet_cnt_sum_60': first_60_percent_unq['retweet_cnt'].sum(),
        'rtw_usr_screen_list_60': tuple(set(first_60_percent['rtw_usr_screen'].tolist())),
        'tw_usr_screen_list_60': tuple(set(first_60_percent['tw_usr_screen'].tolist())),

#         'rt_url': name,
        'retweet_cnt_sum_all': all_rows_unq['retweet_cnt'].sum(),
        'rtw_usr_screen_list_all': tuple(set(all_rows['rtw_usr_screen'].tolist())),
        'tw_usr_screen_list_all': tuple(set(all_rows['tw_usr_screen'].tolist()))
    }
    
    # Append the results for this group to the overall list of results
    results.append(first_60_percent_agg)
#     results.append(all_rows_agg)

# Create a new DataFrame from the results
result_df = pd.DataFrame(results)


In [11]:
pickle_download(Pickle_path+"URL_base_data_train_test.pkl",result_df)

In [12]:
sample_data = result_df[(result_df['retweet_cnt_sum_60']>1000) & (result_df['retweet_cnt_sum_all']>1000) & (result_df['retweet_cnt_sum_all']!=result_df['retweet_cnt_sum_60'])].\
                sort_values(['retweet_cnt_sum_all'],ascending=False)

In [25]:
sample_data

,rt_url,retweet_cnt_sum_60,rtw_usr_screen_list_60,tw_usr_screen_list_60,retweet_cnt_sum_all,rtw_usr_screen_list_all,tw_usr_screen_list_all,usr_screen_60,usr_screen_all
34652,https://support.twitter.com/articles/20169199,123390,"(Theboydrool, CongealedM, pietrohiq, TDarkhand...","(joembayawaphoto, ChocolateWraith, _stylingg, ...",728079,"(Hero2113805826, fatgrlfantasies, VicSanityX, ...","(ChocolateWraith, NickFittXXX, NYGovCuomo, Mas...","(ChocolateWraith, NickFittXXX, Hero2113805826,...","(ChocolateWraith, NickFittXXX, Hero2113805826,..."
47731,https://twitter.com/WFLA/status/12598673542065...,97891,"(TomLazerBum, karenjiggetts, jordieleigh5, Bik...","(Nick__Pulos, QuincyEnunwa, WhomstSaidThat, gr...",100533,"(TomLazerBum, karenjiggetts, JohnCrowell, Des_...","(TedRogersLA, Nick__Pulos, oneunderscore__, ad...","(TomLazerBum, karenjiggetts, Nick__Pulos, jord...","(Des_Shinta, jordieleigh5, oneunderscore__, am..."
41826,https://twitter.com/CNBC/status/12635109230561...,28878,"(AndyLeeParker1, DaultRadio, Vanessa64254192, ...","(PatrioticMills, jaredskolnick, Pat_Thorman, k...",72162,"(ReginaRed4, IBRustyJim, AndyLeeParker1, jetti...","(PatrioticMills, jaredskolnick, Pat_Thorman, h...","(PatrioticMills, jaredskolnick, AndyLeeParker1...","(jaredskolnick, jettisonbaggage, haymarketbook..."
52380,https://twitter.com/realDonaldTrump/status/123...,36124,"(Miss_EmmaMae, l_dermody, WolfWar72757268, Arm...","(JoeBiden, Leslieoo7, freddyatton, markmobility)",36148,"(l_dermody, asher_971, Miss_EmmaMae, ArmandoBr...","(freddyatton, markmobility, AndrewFeinberg, Le...","(l_dermody, freddyatton, markmobility, Leslieo...","(l_dermody, freddyatton, markmobility, AndrewF..."
53155,https://twitter.com/thehill/status/12599411129...,24933,"(Torey_Elwell, bethlastname, Nebby_99)","(mmpadellan, foxxi_loxxi, ewarren)",25121,"(Torey_Elwell, besthealthyou, bethlastname, Ne...","(mmpadellan, foxxi_loxxi, Public_Citizen, ewar...","(Torey_Elwell, foxxi_loxxi, Nebby_99, ewarren,...","(Public_Citizen, Torey_Elwell, foxxi_loxxi, Ne..."
...,...,...,...,...,...,...,...,...,...
48140,https://twitter.com/ajc/status/125624854552037...,1014,"(GeecheeGirlCafe, shayne571, NewAugustWest, Ma...","(theferocity, AngryBlackLady, Wilson__Valdez, ...",1069,"(GeecheeGirlCafe, shayne571, NewAugustWest, Ma...","(ValerieValHalla, theferocity, rtajima, AngryB...","(theferocity, GeecheeGirlCafe, shayne571, NewA...","(shayne571, rtajima, AngryBlackLady, Wilson__V..."
80762,https://www.vanityfair.com/news/2020/05/trump-...,1027,"(BenjaminBaughm2, FGedrich, IDWoods, kernelken2)","(TeaPainUSA, BobKustra, LaurieHosken, leonberg...",1047,"(IDWoods, waterloosunset, BenjaminBaughm2, FGe...","(LaurieHosken, TeaPainUSA, BobKustra, Dr_Shara...","(IDWoods, LaurieHosken, BenjaminBaughm2, FGedr...","(IDWoods, LaurieHosken, waterloosunset, Benjam..."
45396,https://twitter.com/NatashaBertrand/status/126...,1042,"(cgormley9, ohiovoter55)","(brianbeutler, glennkirschner2)",1043,"(KeithTatem, cgormley9, ohiovoter55)","(brianbeutler, SandyGirl4Him, glennkirschner2)","(brianbeutler, cgormley9, glennkirschner2, ohi...","(ohiovoter55, KeithTatem, cgormley9, glennkirs..."
52313,https://twitter.com/puckettwrites/status/12595...,1014,"(park_congress, SLKath, GangstaOma)","(TayAndersonCO, kathyredmond, mmpadellan)",1039,"(park_congress, SLKath, GangstaOma, QueerjohnPA)","(TayAndersonCO, kathyredmond, NC5PhilWilliams,...","(SLKath, GangstaOma, kathyredmond, TayAnderson...","(SLKath, GangstaOma, kathyredmond, TayAnderson..."


In [20]:
pickle_download(Pickle_path+"URL_base_data_train_test_POC.pkl",sample_data)

In [15]:
sample_data['usr_screen_60'] = sample_data.apply(lambda row: tuple(set((row['rtw_usr_screen_list_60']+ row['tw_usr_screen_list_60']))), axis=1)


sample_data['usr_screen_all'] = sample_data.apply(lambda row: tuple(set((row['rtw_usr_screen_list_all']+ row['tw_usr_screen_list_all']))), axis=1)


def extract_user_ids(screen_names, mapping_dict):
    user_ids = []
    for screen_name in screen_names:
        if screen_name in mapping_dict:
            user_ids.append(mapping_dict[screen_name])
    return tuple(user_ids)


sample_data['train_user_ids'] = sample_data['usr_screen_60'].apply(lambda x: extract_user_ids( x, tw_usr_screen_dict))
sample_data['test_user_ids'] = sample_data['usr_screen_all'].apply(lambda x: extract_user_ids(x, tw_usr_screen_dict))


In [625]:
sample_data.head(2)

,rt_url,retweet_cnt_sum_60,rtw_usr_screen_list_60,tw_usr_screen_list_60,retweet_cnt_sum_all,rtw_usr_screen_list_all,tw_usr_screen_list_all,usr_screen_60,usr_screen_all,train_user_ids,test_user_ids
34652,https://support.twitter.com/articles/20169199,123390,"(Treycarey8, Jboogie16705348, 2020Dennison, dv...","(RondaRousey, TrayTheTrade, NBCNews, DaddyMrCa...",728079,"(Jboogie16705348, dvlzura, cruzz_bi, turboanda...","(RondaRousey, SenTedCruz, _stylingg, PeepshowM...","(RondaRousey, Jboogie16705348, dvlzura, SenTed...","(RondaRousey, Jboogie16705348, dvlzura, SenTed...","(82588, 62644, 35605, 8741, 25542, 98694, 3735...","(82588, 62644, 35605, 8741, 25542, 14824, 3083..."
47731,https://twitter.com/WFLA/status/12598673542065...,97891,"(jordieleigh5, ItsMyOpinion16, TomLazerBum, St...","(mvzelenks, WhomstSaidThat, GoldnSweetCheek, A...",100533,"(scraw28, jordieleigh5, ItsMyOpinion16, TomLaz...","(mvzelenks, WhomstSaidThat, BrianPShea, GoldnS...","(mvzelenks, WhomstSaidThat, jordieleigh5, Gold...","(NayrmanBSC, StaceyExplosion, JeffJSays, Nick_...","(53445, 18997, 47250, 58689, 66875, 87987, 801...","(11090, 80158, 62345, 98662, 35126, 25409, 540..."


In [28]:
### Approach 2
'''
For each URL before 21 May 2020, identify when it was first tweeted, put cut off of 1 week and find all ids which tweeted that url within that timeframe

But for Y ( retweet count): Use the retweet count 3 weeks after first occurance of tweet
'''

'\nFor each URL before 21 May 2020, identify when it was first tweeted, put cut off of 1 week and find all ids which tweeted that url within that timeframe\n\nBut for Y ( retweet count): Use the retweet count 3 weeks after first occurance of tweet\n'

In [234]:
Train_data_dict = {}

In [235]:
url_list = list(master_df['rt_url'].unique())

In [236]:
Test_data_dict = {}


In [274]:
for input_url in url_list[:]:
#     input_url = 'https://support.twitter.com/articles/20169199'
    test_urldf = master_df[master_df['rt_url']==input_url]

    retwet_cnt = test_urldf[['tw_usr_screen','retweet_cnt']].drop_duplicates(['tw_usr_screen'])['retweet_cnt'].sum()
    if retwet_cnt<1000:
        continue
    import pandas as pd
    from datetime import datetime
    import pytz

    # Create sample DataFrame
    # Define original and new formats
    original_format = "%a %b %d %H:%M:%S %z %Y"
    new_format = "%Y-%m-%d %H:%M:%S%z"

    # Define conversion function
    def convert_time(original_string):
        dt = datetime.strptime(original_string, original_format).replace(tzinfo=pytz.UTC)
        return dt.strftime(new_format)

    # Apply conversion function to column


    retweetdf_tmp= test_urldf[['rtw_time','rtw_usr_screen']]
    retweetdf_tmp.columns = ['tw_time','tw_usr_screen']
    retweetdf_tmp['tw_time'] = retweetdf_tmp['tw_time'].apply(lambda x: convert_time(x))
    original_format = '%Y-%m-%d'

    # Convert column to datetime objects
    retweetdf_tmp['tw_time'] = retweetdf_tmp['tw_time'].apply(lambda x: datetime.strptime(x[:10], original_format).date())



    tweet_urldf_1 = test_urldf[['tw_time','tw_usr_screen']]
    tweet_urldf_1['tw_time'] = tweet_urldf_1['tw_time'].apply(lambda x : x.date())

    url_all_users = pd.concat([retweetdf_tmp,\
              tweet_urldf_1],axis=0).drop_duplicates()



    min_time = url_all_users['tw_time'].min()

    import datetime

    min_time = url_all_users['tw_time'].min()

    if min_time < datetime.date(2023, 5, 16):
        one_week = datetime.timedelta(weeks=1)
        one_week_after_min = min_time + one_week
        filtered_df = url_all_users[url_all_users['tw_time'] <= one_week_after_min]
#         print(len(filtered_df))
        if len(filtered_df)<2:
            continue
        Train_data_dict[input_url] = {"Users": tuple(set(filtered_df['tw_usr_screen'].tolist())), "Retweet_cnt": retwet_cnt}
    if min_time > datetime.date(2023, 5, 21):
        one_week = datetime.timedelta(weeks=1)
        one_week_after_min = min_time + one_week
        filtered_df = url_all_users[url_all_users['tw_time'] <= one_week_after_min]
#         print(len(filtered_df))
        if len(filtered_df)<2:
            continue
        Test_data_dict[input_url] = {"Users": tuple(set(filtered_df['tw_usr_screen'].tolist())), "Retweet_cnt": retwet_cnt}

    

In [275]:
pickle_download(Pickle_path+"Train_data_dict_1_week_tmp.pkl",Train_data_dict)

In [276]:
User_level_feat_df = pd.read_csv(csv_path+"User_profile.csv")

# assume your DataFrame is named df
tw_usr_screen_dict = User_level_feat_df.reset_index().set_index('tw_usr_screen')['index'].to_dict()


In [278]:
Train_data_dict_new = {}
for key,val in Train_data_dict.items():
    new_vals = val['Users']
    val['Users'] = extract_user_ids( new_vals, tw_usr_screen_dict)
    Train_data_dict_new[key] ={'Users':extract_user_ids( new_vals, tw_usr_screen_dict),"Retweet_cnt":val['Retweet_cnt']}
    

# sample_data['train_user_ids'] = sample_data['usr_screen_60'].apply(lambda x: extract_user_ids( x, tw_usr_screen_dict))
# sample_data['test_user_ids'] = sample_data['usr_screen_all'].apply(lambda x: extract_user_ids(x, tw_usr_screen_dict))


In [280]:

pickle_download(Pickle_path+"Train_data_dict_1_week.pkl",Train_data_dict_new)

In [239]:
pickle_download(Pickle_path+"Test_data_dict_1_week.pkl",Test_data_dict)

In [287]:
url_subset = list(Train_data_dict.keys())

In [291]:
url_data = master_df[master_df['rt_url'].isin(url_subset)]

In [627]:
similarity_dict_idx = {}

for user,adj in similarity_dict.items():
    inner_dict = {}
    for key,val in adj.items():
        inner_dict[tw_usr_screen_dict[key]] = val
    similarity_dict_idx[tw_usr_screen_dict[user]] = inner_dict

In [630]:
Follower_cnt_dict_idx = {}

for key, val in Follower_cnt_dict.items():
    Follower_cnt_dict_idx[tw_usr_screen_dict[key]] = val